**Convolutional Neural Network (CNN) basierend auf die YOLO-Architektur zur Erkennung von deutschen Verkehrszeichen in Echtzeit mit dem GTSRB-Dataset**

Autoren: Laieb Ahmad, Enes Bayar, Mehmet Akif Onur & Mert-Can Öksuz
<br>
Datum: 19.08.2024
<br>
Modul: KI-DL/ML

## **Befehle zum Installieren/Upgraden von Packages** 
Installieren der benötigten Pakete. Falls Bedarf da ist (Inkompatibilität der Packages mit dem Kernel), können diese Befehle direkt im Notenbook ausgeführt werden, um die entsprechenden Packages upzugraden.

In [ ]:
%%bash #Anstatt einzel % vor jedem Command, gesammelt in einer Funktion
pip install numpy --upgrade
pip install pandas --upgrade
pip install scikit-learn --upgrade
pip install scikit-image --upgrade
pip install pillow --upgrade
pip install imbalanced-learn --upgrade
pip install statsmodels --upgrade
pip install matplotlib --upgrade
pip install seaborn --upgrade
pip install tensorflow --upgrade
pip install ultralytics --upgrade

## **Laden von Packages** 
Hier Importieren wir die benötigten Packages/Frameworks, um unser CNN-Modell aufzubauen, anzutrainieren und in den Use-Case überzuleiten.

In [9]:
from ultralytics import YOLO
import os
import numpy as np
import pandas as pd
import cv2
import shutil
import matplotlib
import csv
import torch

## **Übersicht der Versionen von den genutzten Packages** 
Eine Übersicht über die Versionen der einzelnen Packages/Frameworks. Das dient dazu, die Kompitabilität der Packages mit der Python-Version (oder mit der miniconda-Version) zu überwachen.

In [ ]:
print("Package 'Numpy':",np.__version__)
print("Package 'Pandas':",pd.__version__)
print("Package 'OpenCV':",cv2.__version__)
print("Package 'Matplotlib':",matplotlib.__version__)
print("Package 'CSV':",csv.__version__)
print("Package 'PyTorch':",torch.__version__)

## **Installieren von Yolov5** 
Eine Übersicht über die Versionen der einzelnen Packages/Frameworks. Das dient dazu, die Kompitabilität der Packages mit der Python-Version (oder mit der miniconda-Version) zu überwachen.

In [15]:
%%bash
git clone https://github.com/ultralytics/yolov5
cd yolov5
pip install -r requirements.txt

  � r   d a s   W i n d o w s - S u b s y s t e m   f � r   L i n u x   w u r d e n   k e i n e   D i s t r i b u t i o n e n   i n s t a l l i e r t . 
 D i s t r i b u t i o n e n   z u r   I n s t a l l a t i o n   f i n d e n   S i e   i m   M i c r o s o f t   S t o r e : 
 h t t p s : / / a k a . m s / w s l s t o r e 
 

CalledProcessError: Command 'b'git clone https://github.com/ultralytics/yolov5\ncd yolov5\npip install -r requirements.txt\n'' returned non-zero exit status 1.

## **Importieren des Datensatzes** 
Eine Übersicht über die Versionen der einzelnen Packages/Frameworks. Das dient dazu, die Kompitabilität der Packages mit der Python-Version (oder mit der miniconda-Version) zu überwachen.

## **Splitten des Datensatzes in Train und Validation** 
